In [2]:
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

d= load_digits()

In [3]:
df= pd.DataFrame(d.data, columns= d.feature_names)
df['target']= d.target
df
Y= df.target
X= df.drop(['target'], axis= 'columns')

In [22]:
def getscore(model, xtrn,xtst, ytrn, ytst):
    model.fit(xtrn, ytrn)
    return model.score(xtst, ytst)

In [62]:
score={}
models={'Random Forest':RandomForestClassifier(),'Decision Tree': DecisionTreeClassifier(),'SVM': SVC()}
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kf= KFold(n_splits=5)
for a, b in models.items():
    for train_index, test_index in kf.split(X):
        xtrain,xtest,ytrain,ytest = d.data[train_index], d.data[test_index], Y[train_index], Y[test_index]
        score.update( {a : [getscore(models[a],xtrain, xtest, ytrain, ytest)]})
        


score


{'Random Forest': [0.9415041782729805],
 'Decision Tree': [0.7938718662952646],
 'SVM': [0.9415041782729805]}

In [63]:
dict={}
for a, b in models.items():
    dict[a] = cross_val_score(models[a], X, Y, cv= 5)

dict

{'Random Forest': array([0.925     , 0.90833333, 0.95821727, 0.96657382, 0.93036212]),
 'Decision Tree': array([0.79722222, 0.725     , 0.81337047, 0.82729805, 0.78272981]),
 'SVM': array([0.96111111, 0.94444444, 0.98328691, 0.98885794, 0.93871866])}

In [ ]:
# Now Having leanrt That that cross_val_score does exactly ythe same